In [4]:
import pandas as pd 
import numpy as np

In [8]:
# öğrenci verilerinin olduğu csv dosyasını içe aktardım öğrencii veri setinin yapısını görmek için
df = pd.read_csv("student-por.csv")
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [9]:
# geçti mi kaldı mı? 
# kontrol etmek için G3 notu ile birlikte oluşturduğum passed sütununu yan yana gösterdim.
df["passed"] = (df["G3"] >= 10).astype(int)
df[["G3", "passed"]].head()

,G3,passed
0,11,1
1,11,1
2,12,1
3,14,1
4,13,1


In [10]:
# Modelin tahmin yapması için öğrenci bilgileriyle hedef değişkeni ayırmam lazımdı.
# Bu yüzden X’e öğrencinin özelliklerini, y’ye ise geçip geçmediği bilgisini koydum.
X = df.drop(["G1", "G2", "G3", "passed"], axis=1)
y = df["passed"]
# x değişkenini kontrol ettim
X.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,4
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,2
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,6
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,0
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,0


In [11]:
#yazı olan sütunları sayısal hale getirdim modelin anlaması için
X_encoded = pd.get_dummies(X, drop_first=True)
#kontrol ettim son halini
X_encoded.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,True,False,True,False,False,False,True,True,False,False
1,17,1,1,1,2,0,5,3,3,1,...,False,False,False,True,False,False,False,True,True,False
2,15,1,1,1,2,0,4,3,2,2,...,True,False,True,False,False,False,True,True,True,False
3,15,4,2,1,3,0,3,2,2,1,...,True,False,False,True,False,True,True,True,True,True
4,16,3,3,1,2,0,4,3,2,1,...,False,False,False,True,False,False,True,True,False,False


In [13]:
# modeli denemek için veriyi iki parçaya ayırdım 
#bir kısmıyla modeli çalıştırıp, diğer kısmıyla doğru tahmin yapıyor mu diye baktım
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

In [14]:
# geçip geçmeme durumunu tahmin etmek için basit model kulllandım 
#logistic regressionu evet-hayır gibi sonuçlar için uygun olduğundan dolayı seçtim
# modeli de ayırdığım eğitim verisiyle çalıştırdım
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [16]:
#modelin tahminleriyle gerçek sonuçları karşılaştırıp doğruluğunun yüzde kaç olduğuna baktımm
#model %87 doğru oranında tahmin yaptı 
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8769230769230769